In [ ]:
import sys
sys.path.append('../')
from utils import *
from graphUtils import *

# Data Generation

In [ ]:
import pickle
df = pickle.load(open('../../data/imdb/imdb_reviews_1000film.df','rb'))
ground_truth = pickle.load(open('../../data/imdb/imdb_GT.pkl','rb'))
review_ids = pickle.load(open('../../data/imdb/imdb_reviewIDs.pkl','rb'))
row_ids = pickle.load(open('../../data/imdb/imdb_movieIDs.pkl','rb'))
id_revs = inv_map = {v: k for k, v in review_ids.items()}

In [ ]:
import datetime
import csv
movies_dic = {}
with open('../../data/imdb/imdb_movielens.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader)
    for row in csv_reader:
        if row[12].replace('_',' ') not in movies_dic: 
            movies_dic[row[12].replace('_',' ')] = []
            
        temp = [r.replace('_',' ') for r in row[0:10]]
        
        month,year = '',''
        if len(row[10]) > 0:        
            month = datetime.date(1900, int(row[10][4::]), 1).strftime('%B')
            year = row[10][0:4]
        
        temp.append(month.lower() + ' ' + year)
        temp.append(int(float(row[14])))
        
        movies_dic[row[12].replace('_',' ')].append(temp)
        

# Ditto

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
!git clone https://github.com/megagonlabs/ditto
%cd ditto
!pip install -r requirements.txt
!git clone https://github.com/rit-git/Snippext_public
%cd Snippext_public
!pip install -r requirements.txt
%cd ..

import nltk
nltk.download('stopwords')

In [ ]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./
%cd ..

In [ ]:
# some issue with colab
!pip install --upgrade "urllib3==1.25.4" awscli

# Train

In [ ]:
import random
import re



file = open('data/er_magellan/Structured/Beer/train.txt','w')


for movie in [m for m in ground_truth.keys()][0:int(0.6*len(ground_truth))]:
    if movie not in movies_dic: continue
    text = ' '.join(str(m).strip() for m in movies_dic[movie][0] if m not in ['', 'nan'])
    #text = movie + ' ' + ' '.join(str(m).strip() for m in movies_dic[movie][0] if m not in ['', 'nan'])

    text = re.sub('\t',' ',text)


    row = []
    for r in ground_truth[movie]:
        rev = re.sub('\t',' ',review_ids[r])    
        rev = re.sub('\n',' ',rev)
        rev = re.sub(' +',' ',rev)


        file.write('COL movie VAL ' + text + '\tCOL movie VAL ' + rev + '\t1\n' )

        
    for r in random.sample(review_ids.keys(),len(review_ids)):  
        if r not in ground_truth[movie]:
            rev = re.sub('\t',' ',review_ids[r])  
            rev = re.sub('\n',' ',rev)
            rev = re.sub(' +',' ',rev)


            file.write('COL movie VAL ' + text + '\tCOL movie VAL ' + rev + '\t0\n' )


file = open('data/er_magellan/Structured/Beer/valid.txt','w')


for movie in [m for m in ground_truth.keys()][int(0.5*len(ground_truth)):int(0.53*len(ground_truth))]:
    if movie not in movies_dic: continue
    text = ' '.join(str(m).strip() for m in movies_dic[movie][0] if m not in ['', 'nan'])
    #text = movie + ' ' + ' '.join(str(m).strip() for m in movies_dic[movie][0] if m not in ['', 'nan'])

    text = re.sub('\t',' ',text)

    row = []
    for r in ground_truth[movie]:
        rev = re.sub('\t',' ',review_ids[r])    
        rev = re.sub('\n',' ',rev)
        rev = re.sub(' +',' ',rev)

        file.write('COL movie VAL ' + text + '\tCOL movie VAL ' + rev + '\t1\n' )

        
    for r in random.sample(review_ids.keys(),len(review_ids)):  
        if r not in ground_truth[movie]:
            rev = re.sub('\t',' ',review_ids[r])    
            rev = re.sub('\n',' ',rev)
            rev = re.sub(' +',' ',rev)

            file.write('COL movie VAL ' + text + '\tCOL movie VAL ' + rev + '\t0\n' )




file = open('data/er_magellan/Structured/Beer/test.txt','w')


for movie in [m for m in ground_truth.keys()][int(0.55*len(ground_truth)):int(0.6*len(ground_truth))]:
    if movie not in movies_dic: continue
    text = ' '.join(str(m).strip() for m in movies_dic[movie][0] if m not in ['', 'nan'])
    #text = movie + ' ' + ' '.join(str(m).strip() for m in movies_dic[movie][0] if m not in ['', 'nan'])

    text = re.sub('\t',' ',text)

    row = []
    for r in ground_truth[movie]:
        rev = re.sub('\t',' ',review_ids[r])    
        rev = re.sub('\n',' ',rev)
        rev = re.sub(' +',' ',rev)
      
        file.write('COL movie VAL ' + text + '\tCOL movie VAL ' + rev + '\t1\n' )
        
    for r in random.sample(review_ids.keys(),len(review_ids)):  
        if r not in ground_truth[movie]:
            rev = re.sub('\t',' ',review_ids[r])    
            rev = re.sub('\n',' ',rev)
            rev = re.sub(' +',' ',rev)

            file.write('COL movie VAL ' + text + '\tCOL movie VAL ' + rev + '\t0\n' )




In [ ]:
# Step 1:
# train the wdc model with the save_model flag on
# it will produce two files *_dev.pt and *_test.pt which are the 
# best checkpoints on the dev set and the test set.
import time
st = time.time()
!CUDA_VISIBLE_DEVICES=0 python train_ditto.py \
  --task Structured/Beer \
  --batch_size 128 \
  --max_len 128 \
  --lr 3e-5 \
  --n_epochs 2 \
  --finetuning \
  --lm distilbert \
  --fp16 \
  --save_model \
  --da drop_col

print(time.time() - st)

# Test

In [ ]:
import re,json,random
file = open('input/imdb_test.text','w')


for movie in [m for m in ground_truth.keys()][int(0.5*len(ground_truth)):int(1*len(ground_truth))]:
    if movie not in movies_dic: continue
    text = ' '.join(str(m).strip() for m in movies_dic[movie][0] if m not in ['', 'nan'])
    #text = movie + ' ' + ' '.join(str(m).strip() for m in movies_dic[movie][0] if m not in ['', 'nan'])

    text = re.sub('\t',' ',text)

    row = []
    for r in ground_truth[movie]:
        rev = re.sub('\t',' ',review_ids[r])    
        rev = re.sub('\n',' ',rev)
        rev = re.sub(' +',' ',rev)
        l = [{"title" : text}, {"title" : rev}]
        file.write(json.dumps(l) + '\n')
        
    for r in random.sample(review_ids.keys(),800):  
        if r not in ground_truth[movie]:
            rev = re.sub('\t',' ',review_ids[r])    
            rev = re.sub('\n',' ',rev)
            rev = re.sub(' +',' ',rev)
            l = [{"title" : text}, {"title": rev}]
            file.write(json.dumps(l) + '\n')



In [ ]:
!mkdir checkpoints
!mkdir checkpoints/Structured

In [ ]:
!cp Structured_Beer_lm\=distilbert_da\=drop_col_dk\=None_su\=False_size\=None_id\=0_test.pt checkpoints/Structured/Beer.pt

In [ ]:
st = time.time()
!CUDA_VISIBLE_DEVICES=0 python matcher.py \
  --input_path input/imdb_test.text \
  --output_path output/imdb_output.jsonl \
  --lm distilbert \
  --use_gpu \
  --fp16 \
  --checkpoint_path checkpoints/

In [ ]:
from tqdm import tqdm 
file = open('output/imdb_output.jsonl','r')

lines = []
for r in file.readlines():
  lines.append(r)


In [ ]:
from tqdm import tqdm 

ditto_movie = {}

for movie in tqdm([m for m in ground_truth.keys()][int(0.5*len(ground_truth)):int(1*len(ground_truth))],position=0):
    if movie not in movies_dic: continue

    ditto_movie [movie] = {}
    text = ' '.join(str(m).strip() for m in movies_dic[movie][0] if m not in ['', 'nan'])
    #text = movie + ' ' + ' '.join(str(m).strip() for m in movies_dic[movie][0] if m not in ['', 'nan'])

    temp = {}
    for l in lines:
      ll = json.loads(l)

      if ll['left']['title'] == text:
        rev = ll['right']['title']

        if rev not in id_revs: continue
    
        
        if ll['match'] == '0': 
          score = 1 - ll['match_confidence']
        else:
          score = ll['match_confidence']

        temp[id_revs[rev]] = score
      
    ditto_movie[movie] = dict(sorted(temp.items(), key=lambda x: x[1],reverse=True))




In [ ]:
for KK in [1,5,20,500]: 
    i = 0
    precision,recall,fs = 0,0,0
    MAP, MR, hasP = 0,0,0

    for movie in ditto_movie:
        if movie not in ground_truth: continue
        #if row_ids[movie] not in movie_review_d2v: continue
        
        preds =  [f for (f,j) in   sorted(ditto_movie[movie].items(), key=lambda x: x[1],reverse=True)  ][0:KK]

        if len(preds) == 0: continue
        golds = [f for f in ground_truth[movie]]
        i+=1

        MAP += MAP_K(golds,preds)
        MR += MRRR(golds,preds)
        hasP += HAS_POSITIVE(golds,preds)
        
    print('\n#################### ' + str(KK) + ' ###########################\n')
    print('MRR:',MR/i,'MAP:',MAP/i, 'HAS POSITIVE:', hasP/i)
